In [ ]:
from itertools import permutations
from pathlib import Path

import numpy as np

from score_strokes import strokeErrorMatrix
from xmlparse import getXmlScore, minXml

In [ ]:
# Edited from xmlparse.py
def loadRef(han_char, ref_dir = "Reference"):
    stroke_list = []
    frac_dists = []
    ref_path = f"{ref_dir}/{han_char}.han"
    ref_xml = open(ref_path, "r").read()
    root = xmltodict.parse(ref_xml)
    bounds = root["hanDefinition"]["bounds"]
    x_min, y_min, x_max, y_max = (float(bounds["@left"]), float(bounds["@bottom"]), float(bounds["@right"]), float(bounds["@top"]))
    scale = (int(x_max-x_min), int(y_max-y_min))
    strokes = root["hanDefinition"]["strokes"]["stroke"]
    if isinstance(strokes, dict):
        strokes = [strokes]
    for stroke in strokes:
        points = stroke["points"]["forward"]
        point_arr = []
        frac_arr = []
        for point in points["pointDistance"]:
            point_arr.append((float(point["@x"])-x_min,
                               float(point["@y"])-y_min))
            frac_arr.append(float(point["@fractionalDistance"]))
        stroke_list.append(np.array(point_arr))
        frac_dists.append(np.array(frac_arr))
    return stroke_list, frac_dists, scale

# Obtaining scores through heuristic algorithm
def heuristicScores(algo, ref_char, ref_data, char_data):
    heuristic_alignments = []
    heuristic_scores = []
    ref_geometry, ref_progress_percentage, output_size = ref_data
    g_data, _, base_data, stroke_sets, _, f_names = char_data
    for (geometry_length, bases, stroke_set, _, f_name) in zip(g_data, base_data, stroke_sets, _, f_names):
        geometry, progress_percentage = geometry_length
        heuristic_alignment = np.array(algo(geometry, ref_geometry, progress_percentage, ref_progress_percentage))+1
        heuristic_alignments.append(heuristic_alignment)
        heuristic_xml = minXml(ref_char, bases, stroke_set, heuristic_alignment)
        heuristic_score = getXmlScore(heuristic_xml)
        heuristic_scores.append(heuristic_score)
    return heuristic_scores, heuristic_alignments

In [ ]:
# Testing a dynamic programming algorithm

def dynamic(strokes, ref, p_strokes, p_ref):
    error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref)
    stroke_maps = []
    stroke_priority = permutations(range(0, len(ref)-1))
    for _, priority in enumerate(stroke_priority):
        for _, i in priority:
            stroke_map = np.full(len(strokes), -1)
            smallerror = np.argmin(error_maps[i]) # retrieve smallest error for current archetype stroke
            while(stroke_map[smallerror]!=-1):
                # change small error so that we do not repeat over indexes that are already taken
                # just keeps repeating until we land on an index that doesn't already have a value in its place
                error_maps[i][smallerror] = 10000
                smallerror = np.argmin(error_maps[i])
            stroke_map[smallerror] = i
        stroke_maps.append(stroke_map)
    return stroke_maps

ref_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/Reference' # archetype directory
data_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/NewGenes' # gene directory



print(dynamic())

In [ ]:
def compareHeuristic(algo1, algo2):
    return